# Step 2-3 ARIMAX

## Comparison of ARIMAX

ARIMA is autoregressive.

ARIMAX can handle exogenous variables, 

Two experiements for ARIMAX

1. energy mix + core features

2. energy mix + HHI + core features

In [36]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

import os
import warnings

warnings.filterwarnings("ignore")

In [37]:
# Config
TARGET_VARIABLES = 'co2'
CANDIDATE_FEATURES = ['gdp', 'primary_energy_consumption']
SELECTED_COUNTRIES = ['United States', 'China', 'India']
MAX_LAGS = 4
TEST_SIZE = 9

CORE_FEATURES = ['population']

ARIMA_ORDERS = {
    'United States': (0, 1, 0),
    'China': (0, 2, 0),
    'India': (1, 1, 1)
}

# Important features to include
IMPORTANT_FEATURES = [
    'population',
    'gdp',
    'primary_energy_consumption'
    #'Proportions', # These are below
    #'Energy_mix' 
]

HHI_FEATURES = [
    'hhi_detailed',
    'hhi_fossil_comp'
]

PRODUCTION_FEATURES = [
    'coal_production',
    'oil_production',
    'gas_production'
]

CONSUMPTION_FEATURES = [
    'coal_consumption',
    'oil_consumption',
    'gas_consumption',
    'nuclear_consumption',
    'biofuel_consumption',
    'solar_consumption',
    'wind_consumption',
    'hydro_consumption',
    'other_renewables_consumption'
]

SHARE_FEATURES = [
    'coal_share_energy',
    'oil_share_energy',
    'gas_share_energy',
    'nuclear_share_energy',
    'biofuel_share_energy',
    'solar_share_energy',
    'wind_share_energy',
    'hydro_share_energy',
    'other_renewables_share_energy'
]

In [38]:
def mase(y_actual, y_pred, period=1):
    mae_forecast = mean_absolute_error(y_actual, y_pred)

    naive_forecast = y_actual[:-period] if period > 0 else y_actual[:-1]
    actual_for_naive = y_actual[period:] if period > 0 else y_actual[1:]

    if len(naive_forecast) == 0:
        return np.nan
    
    mae_naive = mean_absolute_error(actual_for_naive, naive_forecast)

    if mae_naive == 0:
        return 0 if mae_forecast == 0 else np.inf
    
    return mae_forecast / mae_naive

In [39]:
def load_data(save_dir='data'):
    data_files = {
        'train_3_hhi_detail_fossil': os.path.join(save_dir, 'train_3_hhi_detail_fossil.csv'),
        'test_3_hhi_detail_fossil': os.path.join(save_dir, 'test_3_hhi_detail_fossil.csv'),
        'all_data_df': os.path.join(save_dir, 'all_data_df.csv')
    }

    dfs = {}
    for name, filepath in data_files.items():
        if os.path.exists(filepath):
            dfs[name] = pd.read_csv(filepath)
            print(f"Loaded {name}: {dfs[name].shape}")
        else:
            print(f"{filepath} not found")
    
    return dfs

In [40]:
data = load_data()
train_3_hhi_detail_fossil_df = data['train_3_hhi_detail_fossil']
test_3_hhi_detail_fossil_df = data['test_3_hhi_detail_fossil']
all_data_df = data['all_data_df']

Loaded train_3_hhi_detail_fossil: (135, 1002)
Loaded test_3_hhi_detail_fossil: (27, 1002)
Loaded all_data_df: (55529, 200)


In [41]:
def calculate_pct_change(df, feature_cat, max_lags=MAX_LAGS):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(['country', 'year']).reset_index(drop=True)
    pct_change_cols = []

    for feature in feature_cat:
        if feature not in df_copy.columns:
            continue

        # Pct change on current values
        lag1_col = f"{feature}_lag1"
        if lag1_col in df_copy.columns:
            df_copy[f"{feature}_pct_change"] = ((df_copy[feature] - df_copy[lag1_col]) / df_copy[lag1_col] * 100)
            pct_change_cols.append(f"{feature}_pct_change")

        # Pct change on lagged values
        for lag in range(1, max_lags):
            lag_col = f"{feature}_lag{lag}"
            prev_lag_col = f"{feature}_lag{lag+1}"
            
            if lag_col in df_copy.columns and prev_lag_col in df_copy.columns:
                df_copy[f"{lag_col}_pct_change"] = ((df_copy[lag_col] - df_copy[prev_lag_col]) / df_copy[prev_lag_col] * 100)
                pct_change_cols.append(f"{lag_col}_pct_change")

        # Lag4 for the first row = 0, then shift lag3_pct by country
        last_lag_col = f"{feature}_lag{max_lags}"
        lag3_pct_col = f"{feature}_lag{max_lags-1}_pct_change"

        if last_lag_col in df_copy.columns and lag3_pct_col in df_copy.columns:
            df_copy[f"{last_lag_col}_pct_change"] = df_copy.groupby('country')[lag3_pct_col].shift(1).fillna(0)
            pct_change_cols.append(f"{last_lag_col}_pct_change")
            
        df_copy = df_copy.replace([np.inf, -np.inf], np.nan)
    
    return df_copy, pct_change_cols

In [48]:
train_df = train_3_hhi_detail_fossil_df
test_df = test_3_hhi_detail_fossil_df

print(f"Train years: {train_df['year'].min()} - {train_df['year'].max()}")
print(f"Test years: {test_df['year'].min()} - {test_df['year'].max()}")

Train years: 1969 - 2013
Test years: 2014 - 2022


In [43]:
train_cons_pct, cons_pct_cols = calculate_pct_change(train_df, CONSUMPTION_FEATURES)
train_share_pct, share_pct_cols = calculate_pct_change(train_df, SHARE_FEATURES)

test_cons_pct, _ = calculate_pct_change(test_df, CONSUMPTION_FEATURES)
test_share_pct, _ = calculate_pct_change(test_df, SHARE_FEATURES)

## ARIMAX

In [44]:
def train_ARIMAX(train_country, test_country, exog_features, order):
    train_sorted = train_country.sort_values('year').reset_index(drop=True)
    test_sorted = test_country.sort_values('year').reset_index(drop=True)

    y_train = train_sorted[TARGET_VARIABLES].values
    y_test = test_sorted[TARGET_VARIABLES].values

    available_features = [f for f in exog_features if f in train_sorted.columns]
    
    if not available_features:
        return None
    
    X_train = train_sorted[available_features].values
    X_test = test_sorted[available_features].values
    
    train_mask = ~np.isnan(X_train).any(axis=1) & ~np.isnan(y_train)
    test_mask = ~np.isnan(X_test).any(axis=1) & ~np.isnan(y_test)
    
    X_train = X_train[train_mask]
    y_train = y_train[train_mask]
    X_test = X_test[test_mask]
    y_test = y_test[test_mask]
    
    if len(y_train) == 0 or len(y_test) == 0:
        return None
    
    model = SARIMAX(y_train, exog=X_train, order=order, enforce_stationarity=False, enforce_invertibility=False)
    fitted_model = model.fit(disp=False)

    forecast = fitted_model.forecast(steps=len(y_test), exog=X_test)

    rmse_score = np.sqrt(mean_squared_error(y_test, forecast))
    mase_score = mase(y_test, forecast)

    return {
        'forecast': forecast,
        'actual': y_test,
        'RMSE': rmse_score,
        'MASE': mase_score,
        'n_features': len(available_features),
        'features': available_features
        }

In [ ]:
# Current values, not lags
cons_pct_current = [col for col in cons_pct_cols if '_lag' not in col]
share_pct_current = [col for col in share_pct_cols if '_lag' not in col]

# Combination of feature categories
feature_combs = {
    'Important': IMPORTANT_FEATURES,
    'cons_pct + important': cons_pct_current + IMPORTANT_FEATURES,
    'cons_pct + important + HHI': cons_pct_current + IMPORTANT_FEATURES + HHI_FEATURES,
    'share_pct + important': share_pct_current + IMPORTANT_FEATURES,
    'share_pct + important + HHI': share_pct_current + IMPORTANT_FEATURES + HHI_FEATURES
}

In [59]:
all_results = {}

for country in SELECTED_COUNTRIES:
    print(f"\n{country.upper()}")
    country_results = {}

    # Using optimal orders from ARIMA testing
    order = ARIMA_ORDERS[country]

    for comb_name, features in feature_combs.items():
        print(f"\nTraining {comb_name}")

        if 'cons_pct' in comb_name:
            train_country = train_cons_pct[train_cons_pct['country'] == country]
            test_country = test_cons_pct[test_cons_pct['country'] == country]
            print(f"Using cons_pct dataset")
        elif 'share_pct' in comb_name:
            train_country = train_share_pct[train_share_pct['country'] == country]
            test_country = test_share_pct[test_cons_pct['country'] == country]
            print(f"Using share_pct dataset")
        else:
            train_country = train_df[train_df['country'] == country]
            test_country = test_df[test_df['country'] == country]
            print(f"Using original dataset")

        result = train_ARIMAX(train_country, test_country, features, order)

        if result:
            country_results[comb_name] = result
            print(f"    RMSE: {result['RMSE']:.4f}, MASE: {result['MASE']:.4f}, Features: {result['n_features']}")
        else:
            print(f"    Failed to train")
    
    all_results[country] = country_results


UNITED STATES

Training Important
Using original dataset
    RMSE: 327.7108, MASE: 1.5094, Features: 3

Training cons_pct + important
Using cons_pct dataset
    RMSE: 333.0211, MASE: 1.5071, Features: 11

Training cons_pct + important + HHI
Using cons_pct dataset
    RMSE: 65.1721, MASE: 0.2883, Features: 13

Training share_pct + important
Using share_pct dataset
    RMSE: 308.9255, MASE: 1.3554, Features: 12

Training share_pct + important + HHI
Using share_pct dataset
    RMSE: 92.5064, MASE: 0.4185, Features: 14

CHINA

Training Important
Using original dataset
    RMSE: 89.9612, MASE: 0.2545, Features: 3

Training cons_pct + important
Using cons_pct dataset
    RMSE: 3047.1364, MASE: 11.3003, Features: 11

Training cons_pct + important + HHI
Using cons_pct dataset
    RMSE: 3047.1344, MASE: 11.3003, Features: 13

Training share_pct + important
Using share_pct dataset
    RMSE: 3785.7044, MASE: 10.0601, Features: 12

Training share_pct + important + HHI
Using share_pct dataset
    

In [60]:
summary_data = []

for country in SELECTED_COUNTRIES:
    for combo_name, result in all_results[country].items():
        summary_data.append({
            'Country': country,
            'Feature_Combo': combo_name,
            'RMSE': result['RMSE'],
            'MASE': result['MASE'],
            'N_Features': result['n_features']
        })

summary_df = pd.DataFrame(summary_data)

rmse_pivot = summary_df.pivot(index='Feature_Combo', columns='Country', values='RMSE')
rmse_pivot = rmse_pivot.round(2)
rmse_pivot = rmse_pivot[SELECTED_COUNTRIES]

print("\n\nRMSE by Feature Combination and Country:")
print(rmse_pivot)



RMSE by Feature Combination and Country:
Country                      United States    China   India
Feature_Combo                                              
Important                           327.71    89.96   91.48
cons_pct + important                333.02  3047.14  668.51
cons_pct + important + HHI           65.17  3047.13  208.83
share_pct + important               308.93  3785.70  975.82
share_pct + important + HHI          92.51  3785.71  975.77


Visualisation